<a href="https://colab.research.google.com/github/kdrm12/KP-Nutrifood-FakeNews/blob/main/Stage_1_Data_Preprocessing_(Fake_News).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

Data yang digunakan pada *preprocessing* adalah [Fake and real News Dataset](https://www.kaggle.com/code/ohseokkim/fake-news-easy-nlp-text-classification/data).

# **Download Libraries**

In [ ]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install demoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.4 MB/s 


In [ ]:
pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 14.9 MB/s 
     |████████████████████████████████| 287 kB 65.2 MB/s 


In [ ]:
pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 14.7 MB/s 


In [ ]:
pip install num2words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 116 kB 14.9 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=a3612b16444a430862b1900ef9b084a5184aa98de200e909abf97e9f6d27ca23
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [ ]:
pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.7 MB 14.0 MB/s 


# **Import Libraries**

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import demoji
import contractions
import unidecode
from collections import Counter
from num2words import num2words
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
from spellchecker import SpellChecker
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

pd.options.display.max_columns=None
pd.options.display.max_rows=None
pd.options.display.max_colwidth=None

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# **Load Data**

In [ ]:
dwn_url='https://drive.google.com/uc?id=' + '1zKSY4d2v6Wf3UUhmd-5d-er30PBMXzGg'
fake_df = pd.read_csv(dwn_url)
dwn_url='https://drive.google.com/uc?id=' + '1mXmP3ystGW_ALB2XZQPfIzq3xdQOh0wm'
true_df = pd.read_csv(dwn_url)

In [ ]:
fake_df["output"] = 0
true_df["output"] = 1

df = pd.concat([fake_df, true_df])
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
print(f'Shape of the whole data is: {df.shape[0]} rows and {df.shape[1]} columns')

Shape of the whole data is: 44898 rows and 5 columns


In [ ]:
df.head()

title  \
0                                  WATCH: These Videos Will Make Trump The Most Toxic Candidate EVER   
1                 Here’s What Happens When Children Win A Science Contest In Trump’s America (VIDEO)   
2   TX Teacher Infuriates Mother With Note Implying Her Daughter May Die In Next Orlando-Like Attack   
3                            Japan weighs plans to deal with North Korean evacuees in case of crisis   
4                    "Make Republicans Whole Again!" A divided party struggles to rally behind Trump   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

# **Pre Processing Data**

## Lower Casing

In [ ]:
df.title = df.title.str.lower()
df.text = df.text.str.lower()
df.subject = df.subject.str.lower()

## Remove

### Rare words

In [ ]:
def rare_words(text):
    # tokenization
    tokens = word_tokenize(text)
    counter = Counter()
    for word in tokens:
        counter[word]= +1

    RareWords = []
    number_rare_words = 1
    # take top 1 rare words
    frequentWords = counter.most_common()
    for (word, word_count) in frequentWords[:-number_rare_words-1:-1]:
        RareWords.append(word)

    return RareWords

def remove_rw(text, RareWords):
    tokens = word_tokenize(text)
    without_rw = []
    for word in tokens:
        if word not in RareWords:
            without_rw.append(word)

    without_rw = ' '.join(without_rw)
    return without_rw

### Other

In [ ]:
def remove(text):
  # remove url's
  text = re.sub(r'https?://\S+|www\.\S+', '', text)

  # remove email
  text = re.sub(r'\S+@\S+', '', text)

  # remove Hashtags and Mentions
  text = re.sub(r'(@\S+|#\S+)', '', text)

  # remove date
  ## 1. Remove date formats like: dd/mm/yy(yy), dd-mm-yy(yy), dd(st|nd|rd).mm/yy(yy)
  text = re.sub(r'\d{1,2}(st|nd|rd|th)?[-./]\d{1,2}[-./]\d{2,4}', '', text)
  ## 2. Remove date formats like: 20 apr 21, April 15th, 11th of April, 2021
  text = re.sub(r'(\d{1,2})?(st|nd|rd|th)?[-./,]?\s?(of)?\s?([J|j]an(uary)?|[F|f]eb(ruary)?|[Mm]ar(ch)?|[Aa]pr(il)?|[Mm]ay|[Jj]un(e)?|[Jj]ul(y)?|[Aa]ug(ust)?|[Ss]ep(tember)?|[Oo]ct(ober)?|[Nn]ov(ember)?|[Dd]ec(ember)?)\s?(\d{1,2})?(st|nd|rd|th)?\s?[-./,]?\s?(\d{2,4})?', r'', text)

  # remove html tags
  text = BeautifulSoup(text).get_text()

  # remove emoji
  emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                              u"\U00002500-\U00002BEF"  # chinese char
                              u"\U00002702-\U000027B0"
                              u"\U00002702-\U000027B0"
                              u"\U000024C2-\U0001F251"
                              u"\U0001f926-\U0001f937"
                              u"\U00010000-\U0010ffff"
                              u"\u2640-\u2642"
                              u"\u2600-\u2B55"
                              u"\u200d"
                              u"\u23cf"
                              u"\u23e9"
                              u"\u231a"
                              u"\ufe0f"  # dingbats
                              u"\u3030"
                              "]+", flags=re.UNICODE)
  text = emoji_pattern.sub(r'', text)

  # remove puctuations
  PUNCTUATIONS = string.punctuation
  text = text.translate(str.maketrans('', '', PUNCTUATIONS))

  # remove stopword
  STOPWORDS = set(stopwords.words('english'))
  text = ' '.join([word for word in text.split() if word not in STOPWORDS])

  # remove number
  text = re.sub(r'\d+', '',text)

  # remove whitespace
  text = " ".join(text.split())

  # remove rare word
  text = remove_rw(text, rare_words(text))

  return text


### Remove all of them

In [ ]:
# run function remove
df.text = df.text.apply(lambda x: remove(x))

In [ ]:
df.head()

title  \
0                                  watch: these videos will make trump the most toxic candidate ever   
1                 here’s what happens when children win a science contest in trump’s america (video)   
2   tx teacher infuriates mother with note implying her daughter may die in next orlando-like attack   
3                            japan weighs plans to deal with north korean evacuees in case of crisis   
4                    "make republicans whole again!" a divided party struggles to rally behind trump   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

## Stemming

In [ ]:
stemmer = PorterStemmer()

def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

In [ ]:
df_steammed = df.copy()
df_steammed.text = df_steammed.text.apply(lambda text: stem_words(text))

## Convert accented characters to ASCII characters



In [ ]:
def accented_to_ascii(text):
    return unidecode.unidecode(text)

In [ ]:
df_steammed.text = df_steammed.text.apply(lambda x: accented_to_ascii(x))

# **Delete NaN text data**

In [ ]:
# missing_values_count = df_steammed.isnull().sum() # jumlah data yang hilang per kolom
# print("Number of missing data points per column:\n")
# print (missing_values_count)

df_steammed.dropna(inplace=True)

# print(df_steammed.isna().sum(),'\n')

In [ ]:
# Cek lagi
# Mengecek nilai null
# print(df_steammed.isna().sum(),'\n')

#Total missing value
# print('Total NaN = ',df_steammed.isna().sum().sum())

# **Splitting Dataset**

In [ ]:
train, test = train_test_split(df_steammed, test_size=0.3, random_state=35)

# **Save clean data to csv**

In [ ]:
train.to_csv('train_fake_and_real_news.csv', index=False)
test.to_csv('test_fake_and_real_news.csv', index=False)

# **Download clean data**

In [ ]:
from google.colab import files
files.download('train_fake_and_real_news.csv')
files.download('test_fake_and_real_news.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>